# Compare particle runs

In this notebook, I compare the different Parcels output files, with the primary goal of analysing what the most efficient way is to decrease the number of particles 'stuck' on the coral. All output files are preceded by `'d2c'`, so I search the output folders for files containing this prefix, read the files and extract the metadata into a pandas dataframe. This dataframe can then be used for analysis.

In [1]:
import pandas as pd
import re
import xarray as xr
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib as mpl
from matplotlib.colors import ListedColormap
import cmocean

plt.style.use('ggplot')

In [ ]:
sdata = data.where(data['finaldistance'][:,-1]==0,drop=True)
bdata = data.where(data['closestobject'][:,-1]==len(objects),drop=True)

fig = plt.figure()
ax = plt.axes(xticks=[],yticks=[])
ax.set_title('Dashboard '+dfilename)
ax.text(0.05,0.95,'Flow: '+flow)
ax.text(0.05,0.9,'Time direction: '+fb)
ax.text(0.05,0.85,'Runtime: '+str(runtime.seconds)+' seconds')
ax.text(0.05,0.8,'Execution dt: '+str(dt.total_seconds())+' seconds')
ax.text(0.05,0.75,'Output dt: '+str(outputdt.total_seconds())+' seconds')
ax.text(0.05,0.7,'dx/dy: '+str(dx)+' m')
ax.text(0.05,0.65,'Number of particles released: '+str(len(data['traj'])))
ax.text(0.05,0.6,'Number of particles beached: '+str(len(bdata['traj'])))
ax.text(0.05,0.55,'Percentage of particles beached: '+str(np.round(len(bdata['traj'])/len(data['traj'])*100,1))+' %')
# plt.savefig('Figures/Dashboard-'+flow+fb)

Run the next cell to create the dataframe and save it using pandas' `to_pickle`. If you want to analyse a saved dataframe, skip this cell.

In [7]:
output = pd.DataFrame(columns=['flow','dx','time direction','dt','runtime','execution time','output dt','#particles','#stuck','%stuck','#beached','%beached','beaching scheme'])

foldernames = ['16objects','21objects']
    
for folder in foldernames:
    objects = np.load(folder+'/preprocessed/objects.npy')
    for file in os.listdir(folder+'/pfiles'):
            data = xr.open_dataset(folder+'/pfiles/'+file)
            sdata = data.where(data['finaldistance'][:,-1]==0,drop=True)
            bdata = data.where(data['closestobject'][:,-1]==len(objects),drop=True)
            filelist = file.split('-')
            flow = filelist[-3]
            fb = filelist[-1][:-3]
            
            filename = flow+'.nc'
            flowdata = xr.open_dataset(folder+'/'+filename)
            
            dx = flowdata['X'][1]-flowdata['X'][0]
            dx = dx.values

            outputdtlist = data.attrs['outputdt'].split()
            outputdt = float(outputdtlist[0])

            dtlist = data.attrs['dt'].split()
            dt = float(dtlist[0])

            runtimelist = data.attrs['runtime'].split()
            runtime = float(runtimelist[0])

            if 'executiontime' in data.attrs:
                exectimelist = data.attrs['executiontime'].split()
                exectime = float(exectimelist[0])
            else:
                exectime = None

            if 'beaching_strategy' in data.attrs:
                strategy = data.attrs['beaching_strategy']
            else:
                strategy = 0


            df = pd.DataFrame([[flow,dx,fb,dt,runtime,exectime,outputdt,len(data['traj']),len(sdata['traj']),np.round(len(sdata['traj'])/len(data['traj'])*100,2),len(bdata['traj']),np.round(len(bdata['traj'])/len(data['traj'])*100,2),strategy]],columns=['flow','dx','time direction','dt','runtime','execution time','output dt','#particles','#stuck','%stuck','#beached','%beached','beaching scheme'])
            frames = [output,df]
            output = pd.concat(frames)

output.to_pickle('RunsComp.pkl')

In [2]:
output = pd.read_pickle('RunsComp.pkl')

In [13]:
dx.values

array(0.015625)

In [8]:
# comp = output.where(abs(output['dt']) == 0.01)
# comp = output.where(output['flow'] == 'waveparabolic')
# output = comp.dropna() 
output

,flow,dx,time direction,dt,runtime,execution time,output dt,#particles,#stuck,%stuck,#beached,%beached,beaching scheme
0,parabolic,<xarray.DataArray 'X' ()>\narray(0.015625),forward,0.001,60.0,846.331431,0.1,33324,96,0.29,367,1.10,0
0,parabolic,<xarray.DataArray 'X' ()>\narray(0.015625),forward,0.010,60.0,216.902402,0.1,33324,155,0.47,396,1.19,0
0,uniform,<xarray.DataArray 'X' ()>\narray(0.015625),forward,0.001,60.0,1038.460360,0.1,33324,167,0.50,331,0.99,0
0,uniform,<xarray.DataArray 'X' ()>\narray(0.015625),forward,0.010,60.0,242.889440,0.1,33324,271,0.81,349,1.05,0
0,waveparabolic,<xarray.DataArray 'X' ()>\narray(0.015625),forward,0.001,60.0,651.310856,0.1,33324,212,0.64,347,1.04,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,parabolic,<xarray.DataArray 'X' ()>\narray(0.015625),forward,0.001,60.0,1052.788042,0.1,38464,0,0.00,0,0.00,3
0,uniform,<xarray.DataArray 'X' ()>\narray(0.015625),forward,0.001,60.0,461.874729,0.1,38464,0,0.00,0,0.00,3
0,waveparabolic,<xarray.DataArray 'X' ()>\narray(0.015625),forward,0.001,60.0,477.465188,0.1,38464,0,0.00,0,0.00,3
0,parabolic,<xarray.DataArray 'X' ()>\narray(0.015625),forward,0.001,30.0,4714.441505,0.1,221221,237,0.11,2,0.00,2


In [14]:
my_cmap = ListedColormap(['dodgerblue','g','purple','darkorange','red','purple'])

In [15]:
%matplotlib qt
dx_cmap = plt.get_cmap('cmo.matter',len(np.unique(output['beaching scheme'])))

fig = plt.figure()
ax = plt.axes()
ax.set_title('Beaching')
ax.set_ylabel('Particles beached [%]')
ax.set_xlabel('dt [s]')
sc0 = ax.scatter(output['dt'][output['beaching scheme']==0],output['%beached'][output['beaching scheme']==0],c = [my_cmap(0)],alpha=0.7,label='0',edgecolors='k')
sc1 = ax.scatter(output['dt'][output['beaching scheme']==1],output['%beached'][output['beaching scheme']==1],c = [my_cmap(1)],alpha=0.7,label='1',edgecolors='k')
sc2 = ax.scatter(output['dt'][output['beaching scheme']==2],output['%beached'][output['beaching scheme']==2],c = [my_cmap(2)],alpha=0.7,label='2',edgecolors='k',s=50,marker='^')
sc3 = ax.scatter(output['dt'][output['beaching scheme']==3],output['%beached'][output['beaching scheme']==3],c = [my_cmap(3)],alpha=0.7,label='3',edgecolors='k')
# sc4 = ax.scatter(output['dt'][output['beaching scheme']==4],output['%beached'][output['beaching scheme']==4],c = [my_cmap(4)],label='4',edgecolors='k')
# sc5 = ax.scatter(output['dt'][output['beaching scheme']==5],output['%beached'][output['beaching scheme']==5],c = [my_cmap(5)],label='5',edgecolors='k')
ax.legend(title = 'beaching scheme')
ax.set_xscale('log')
ax.set_xlim(left=0.00005)
ax.set_ylim(top=10)
# ax.set_yscale('log')
# plt.savefig('Figures/runcomp-beaching-dtdx')

(-0.5012926209607316, 10)

In [17]:
%matplotlib qt

fig = plt.figure()
ax = plt.axes()
ax.set_title('Stuck')
ax.set_ylabel('Particles stuck [%]')
ax.set_xlabel('dt [s]')
sc0 = ax.scatter(output['dt'][output['beaching scheme']==0],output['%stuck'][output['beaching scheme']==0],c = [my_cmap(0)],alpha=0.7,label='0',edgecolors='k')
sc1 = ax.scatter(output['dt'][output['beaching scheme']==1],output['%stuck'][output['beaching scheme']==1],c = [my_cmap(1)],alpha=0.7,label='1',edgecolors='k')
sc2 = ax.scatter(output['dt'][output['beaching scheme']==2],output['%stuck'][output['beaching scheme']==2],c = [my_cmap(2)],alpha=0.7,label='2',edgecolors='k',s=50,marker='^')
sc3 = ax.scatter(output['dt'][output['beaching scheme']==3],output['%stuck'][output['beaching scheme']==3],c = [my_cmap(3)],alpha=0.7,label='3',edgecolors='k')
# sc4 = ax.scatter(output['dt'][output['beaching scheme']==4],output['%stuck'][output['beaching scheme']==4],c = [my_cmap(4)],label='4',edgecolors='k')
ax.legend(title='beaching scheme',loc = 'upper center')
ax.set_xscale('log')
ax.set_xlim(left=0.00005)
ax.set_ylim(bottom=0)
ax.set_ylim(top=10)
# plt.savefig('Figures/runcomp-stuck-dtdx')

(0.0, 10)

In [20]:
%matplotlib qt

fig = plt.figure()
ax = plt.axes()
ax.set_title('Beached vs Stuck')
ax.set_ylabel('Particles beached [%]')
ax.set_xlabel('Particles stuck [%]')
x = np.linspace(0,5)
y = x
ax.plot(x,y,label='beached = stuck')
sc0 = ax.scatter(output['%stuck'][output['beaching scheme']==0],output['%beached'][output['beaching scheme']==0],c = [my_cmap(0)],edgecolors='k',alpha=0.7,label='0')
sc1 = ax.scatter(output['%stuck'][output['beaching scheme']==1],output['%beached'][output['beaching scheme']==1],c = [my_cmap(1)],edgecolors='k',alpha=0.7,label='1')
sc2 = ax.scatter(output['%stuck'][output['beaching scheme']==2],output['%beached'][output['beaching scheme']==2],c='w',edgecolors='k',alpha=0.7,label='2')
sc3 = ax.scatter(output['%stuck'][output['beaching scheme']==3],output['%beached'][output['beaching scheme']==3],c = [my_cmap(3)],edgecolors='k',alpha=0.7,label='3')
# sc4 = ax.scatter(output['%stuck'][output['beaching scheme']==4],output['%beached'][output['beaching scheme']==4],c='yellow',edgecolors='k',alpha=0.7,label='4')
# ax.set_ylim(bottom=0)
# ax.set_xlim(left=0)
ax.legend(title = 'beaching scheme')
# plt.savefig('Figures/runcomp-beached-vs-stuck')

In [21]:
%matplotlib qt

fig = plt.figure()
ax = plt.axes()
ax.set_title('Beached vs Stuck')
ax.set_ylabel('Particles beached/stuck [%]')
ax.set_xlabel('Beaching scheme')

sc0 = ax.scatter(output['beaching scheme'][output['dt']==0.01],output['%beached'][output['dt']==0.01],c = 'orangered',edgecolors='k',alpha=0.7,label='0.01')
sc1 = ax.scatter(output['beaching scheme'][output['dt']==0.001],output['%beached'][output['dt']==0.001],c = 'deepskyblue',edgecolors='k',alpha=0.7,label='0.001')
sc2 = ax.scatter(output['beaching scheme'][output['dt']==0.0001],output['%beached'][output['dt']==0.0001],c = 'greenyellow',edgecolors='k',alpha=0.7,label='0.0001')
# sc0 = ax.scatter(output['beaching scheme'][output['dt']==0.01],output['%stuck'][output['dt']==0.01],c = 'orangered',marker='^',edgecolors='k',alpha=0.7,label='0')
# sc1 = ax.scatter(output['beaching scheme'][output['dt']==0.001],output['%stuck'][output['dt']==0.001],c = 'deepskyblue',marker='^',edgecolors='k',alpha=0.7,label='1')
ax.legend(title = 'dt')